In [6]:
import re
from docx import Document
import os

# 打开文档
docx_path = r"C:\Users\Yancey\Downloads\Tencent_QQ_File\乡村教师极简智能教学20法（初稿） (1).docx"
doc = Document(docx_path)

# 输出目录
output_dir = "images"
os.makedirs(output_dir, exist_ok=True)

# 存储编号 → 图片序号映射
img_count = 1
current_numbering = "0"

for i, para in enumerate(doc.paragraphs):
    # 遍历段落 runs，提取图片
    for run in para.runs:
        drawing = run._element.xpath('.//a:blip')
        for blip in drawing:
            rId = blip.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
            image_part = doc.part.related_parts[rId]
            image_bytes = image_part.blob

            # 默认文件名（如果没找到“图X.X”）
            filename = f"{current_numbering}_img{img_count}.png"

            # 检查下一段是否有“图编号”
            if i + 1 < len(doc.paragraphs):
                next_text = doc.paragraphs[i + 1].text.strip()
                # 宽松匹配：允许 "图" 后面有 0~1 个空格，编号中间可有空格
                m = re.match(r"^图\s*([\d\.]+)", next_text)
                if m:
                    current_numbering = m.group(1)  # 提取 "1.2" 这样的编号
                    filename = f"{current_numbering}.png"

            filepath = os.path.join(output_dir, filename)
            with open(filepath, "wb") as f:
                f.write(image_bytes)

            print(f"保存 {filepath}")
            img_count += 1


保存 images\0_img1.png
保存 images\0_img2.png
保存 images\0_img3.png
保存 images\0_img4.png
保存 images\1.1.png
保存 images\1.2.png
保存 images\1.3.png
保存 images\1.4.png
保存 images\1.5.png
保存 images\1.6.png
保存 images\1.7.png
保存 images\1.8.png
保存 images\1.9.png
保存 images\1.9_img14.png
保存 images\1.9_img15.png
保存 images\2.1.png
保存 images\2.2.png
保存 images\2.3.png
保存 images\2.4.png
保存 images\2.5.png
保存 images\2.6.png
保存 images\2.7.png
保存 images\2.8.png
保存 images\2.9.png
保存 images\2.10.png
保存 images\2.11.png
保存 images\2.12.png
保存 images\2.13.png
保存 images\2.14.png
保存 images\2.15.png
保存 images\2.15.png
保存 images\2.16.png
保存 images\2.18.png
保存 images\2.18.png
保存 images\2.17.png
保存 images\2.17_img36.png
保存 images\2.17_img37.png
保存 images\3.1.png
保存 images\3.2.png
保存 images\3.3.png
保存 images\3.4.png
保存 images\3.5.png
保存 images\3.6.png
保存 images\3.7.png
保存 images\3.8.png
保存 images\3.9.png
保存 images\3.10.png
保存 images\3.11.png
保存 images\3.12.png
保存 images\3.13.png
保存 images\3.14.png
保存 images\3.15.png
保存 images

In [9]:
import subprocess
import re

def get_fingerprint(keystore, alias, storepass, algo="SHA1"):
    """
    调用 keytool 获取 SHA1 或 SHA256 指纹
    """
    try:
        cmd = [
            "keytool", "-list", "-v",
            "-keystore", keystore,
            "-alias", alias,
            "-storepass", storepass,
            "-storetype", "JKS",
            "-J-Djdk.certpath.disabledAlgorithms="
        ]

        # 不用 text=True，改用 universal_newlines=True，更稳
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

        # 自动检测编码（Win 默认 GBK）
        output = result.stdout
        if not output:
            output = result.stderr

        print("========== keytool 输出 ==========")
        print(output)
        print("=================================")

        if result.returncode != 0:
            print("❌ keytool 执行失败:", result.stderr)
            return None

        # 正则匹配指纹
        match = re.search(rf"{algo}:\s*([0-9A-F:]+)", output, re.IGNORECASE)
        if match:
            return match.group(1).replace(":", "").upper()
        else:
            print(f"⚠️ 未找到 {algo} 指纹")
            return None

    except Exception as e:
        print("❌ 出现异常:", e)
        return None


if __name__ == "__main__":
    keystore = r"C:\Users\Yancey\Desktop\2\HUBUHelper\my-release-key.jks"
    alias = "my-key-alias"
    storepass = "597994"

    sha1 = get_fingerprint(keystore, alias, storepass, "SHA1")
    sha256 = get_fingerprint(keystore, alias, storepass, "SHA256")

    if sha1:
        print("✅ SHA1 指纹:", sha1)
    if sha256:
        print("✅ SHA256 指纹:", sha256)


========== keytool 输出 ==========
别名: my-key-alias
创建日期: 2025年10月8日
条目类型: PrivateKeyEntry
证书链长度: 1
证书[1]:
所有者: CN=Yancey, OU=YanceyStudio, O=YanceyStudio, L=WUHAN, ST=HUBEI, C=ZH
发布者: CN=Yancey, OU=YanceyStudio, O=YanceyStudio, L=WUHAN, ST=HUBEI, C=ZH
序列号: 52ef6db8e55d3087
生效时间: Wed Oct 08 22:16:48 CST 2025, 失效时间: Sun Feb 23 22:16:48 CST 2053
证书指纹:
	 SHA1: D4:DE:42:3D:F0:DA:48:6F:EE:B2:8D:83:1C:25:14:E6:BC:2C:11:C2
	 SHA256: 7B:0B:C6:AB:5B:B8:BE:83:C8:87:2E:BB:2F:12:7C:88:9A:D6:89:71:6D:AC:24:F0:F6:CF:38:6C:CF:44:B6:1D
签名算法名称: SHA384withRSA
主体公共密钥算法: 2048 位 RSA 密钥
版本: 3

扩展: 

#1: ObjectId: 2.5.29.14 Criticality=false
SubjectKeyIdentifier [
KeyIdentifier [
0000: 1B 06 E6 B9 62 1B 99 EF   67 E0 E0 61 43 CA 61 4C  ....b...g..aC.aL
0010: 3A 40 08 14                                        :@..
]
]


========== keytool 输出 ==========
别名: my-key-alias
创建日期: 2025年10月8日
条目类型: PrivateKeyEntry
证书链长度: 1
证书[1]:
所有者: CN=Yancey, OU=YanceyStudio, O=YanceyStudio, L=WUHAN, ST=HUBEI, C=ZH
发布者: CN=Yancey, 